In [1]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
import gradio

In [2]:
(x_train, y_train),(x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [3]:
x_train.shape

(60000, 28, 28)

In [4]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [5]:
model.fit(x_train, y_train, epochs=1, steps_per_epoch=3)

Epoch 1/1
3/3 [==============================] - 8s 3s/step - loss: 2.0548 - acc: 0.3412


In [6]:
inp = gradio.inputs.ImageUpload(image_width=28, image_height=28, num_channels=None, image_mode='L')

In [7]:
iface = gradio.Interface(inputs=inp, outputs="label", model=model, model_type='keras')

In [8]:
iface.launch(share=False)

NOTE: Gradio is in beta stage, please report all bugs to: a12d@stanford.edu
Model available locally at: http://localhost:7860/interface.html
To create a public link, set `share=True` in the argument to `launch()`


('http://localhost:7860/interface.html', None)

127.0.0.1 - - [05/Mar/2019 23:13:12] "GET /interface.html HTTP/1.1" 200 -
127.0.0.1 - - [05/Mar/2019 23:13:12] "GET /static/js/all-io.js HTTP/1.1" 200 -


{'label': 2, 'confidences': [{'label': 2, 'confidence': 1.0}, {'label': 0, 'confidence': 0.0}, {'label': 0, 'confidence': 0.0}]}


127.0.0.1 - - [05/Mar/2019 23:14:47] "GET /interface.html HTTP/1.1" 200 -
127.0.0.1 - - [05/Mar/2019 23:14:47] "GET /static/css/style.css HTTP/1.1" 200 -
127.0.0.1 - - [05/Mar/2019 23:14:47] "GET /static/css/gradio.css HTTP/1.1" 200 -
127.0.0.1 - - [05/Mar/2019 23:14:47] "GET /static/js/utils.js HTTP/1.1" 200 -
127.0.0.1 - - [05/Mar/2019 23:14:47] "GET /static/js/all-io.js HTTP/1.1" 200 -
127.0.0.1 - - [05/Mar/2019 23:14:47] "GET /static/js/image-upload-input.js HTTP/1.1" 200 -
127.0.0.1 - - [05/Mar/2019 23:14:47] "GET /static/img/logo_inline.png HTTP/1.1" 200 -
127.0.0.1 - - [05/Mar/2019 23:14:47] "GET /static/js/class-output.js HTTP/1.1" 200 -
127.0.0.1 - - [05/Mar/2019 23:14:48] code 404, message File not found
127.0.0.1 - - [05/Mar/2019 23:14:48] "GET /favicon.ico HTTP/1.1" 404 -


{'label': 2, 'confidences': [{'label': 2, 'confidence': 1.0}, {'label': 0, 'confidence': 0.0}, {'label': 0, 'confidence': 0.0}]}


127.0.0.1 - - [05/Mar/2019 23:15:29] "GET /interface.html HTTP/1.1" 200 -
127.0.0.1 - - [05/Mar/2019 23:15:29] "GET /static/js/all-io.js HTTP/1.1" 200 -
127.0.0.1 - - [05/Mar/2019 23:15:32] "GET /interface.html HTTP/1.1" 200 -


{'label': 2, 'confidences': [{'label': 2, 'confidence': 1.0}, {'label': 0, 'confidence': 0.0}, {'label': 0, 'confidence': 0.0}]}


127.0.0.1 - - [05/Mar/2019 23:18:47] "GET /interface.html HTTP/1.1" 200 -
127.0.0.1 - - [05/Mar/2019 23:18:47] "GET /static/css/gradio.css HTTP/1.1" 200 -
127.0.0.1 - - [05/Mar/2019 23:18:47] "GET /static/js/utils.js HTTP/1.1" 200 -
127.0.0.1 - - [05/Mar/2019 23:18:47] "GET /static/js/all-io.js HTTP/1.1" 200 -
127.0.0.1 - - [05/Mar/2019 23:18:47] "GET /static/js/image-upload-input.js HTTP/1.1" 200 -
127.0.0.1 - - [05/Mar/2019 23:18:48] "GET /static/js/class-output.js HTTP/1.1" 200 -


{'label': 2, 'confidences': [{'label': 2, 'confidence': 1.0}, {'label': 0, 'confidence': 0.0}, {'label': 0, 'confidence': 0.0}]}
